# naver_movie_commot_classifier/nsmc.ipynb

네이버영화 댓글 감성분석

- https://github.com/e9t/nsmc

In [1]:
!uv pip install requests pandas scikit-learn 

Using Python 3.12.12 environment at: C:\Users\Playdata\Documents\SKN21\08_NLP-자연어처리\.venv
Audited 3 packages in 73ms


In [6]:
import requests
from io import StringIO
import pandas as pd
url = "https://raw.githubusercontent.com/e9t/nsmc/refs/heads/master/ratings.txt"
res = requests.get(url)
if res.status_code == 200:
    str_id = StringIO(res.text) # 메모리의 문자열(str)으로부터 값을 읽을 수 있는 InputStream
    df = pd.read_csv(str_id, sep="\t")


In [7]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [9]:
df['label'].value_counts()

label
1    100000
0    100000
Name: count, dtype: int64

In [10]:
df.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [12]:
# 결측치 제거
df = df.dropna()
df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [21]:
# 중복된 댓글(document) 확인
df[df.duplicated(subset=["document"], keep=False)].sort_values('document')
# keep : False - 모든 중복행들을 True, "first", "last" (중복된 것들 중 첫번째/마지막 것만 True)

,id,document,label


In [ ]:
# 중복된 댓글이 있는 행은 하나만 남기고 제거
df = df.drop_duplicates(subset='document')

In [22]:
df.shape

(194543, 3)

In [23]:
df.reset_index(drop=True, inplace=True)

In [ ]:
# 네이버 댓글 토큰화 처리 클래스
# 1. 다운로드
# 2. 기본적인 전처리 진행
# 3. 형태소기반 토큰화 진행
import os
import requests
import pandas as pd
from kiwipiepy import Kiwi

class NSMCTokenizer:

    def __init__(self, save_path:str):
        # save_path : 데이터셋 다운 후 저장할 디렉토리 경로.

Exception: Cannot open extract.mdl for WordDetector